# ATMOS 2022 - CHIRTS Analysis

## 0. Import Libraries

In [6]:
import xarray as xr
from utils.data import *
from utils.utils import *
import matplotlib.pyplot as plt
import seaborn as sns
import proplot

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
# numbers starting with 2 or 3: urban
# numbers starting with 1 except 10: rural
# number 10: water
urban_tiles = [21,22,23,30]
rural_tiles = [11,12,13]

## 1. Number of Days Analysis

In [49]:
temp_threshold = 30
lu_year = 2015 # 2000, 2015

In [50]:
provinces = ['istanbul', 'ankara']

# land uses
luses = ['urban', 'nourban']

data_days_dict = {}
for province in provinces:
    
    luse_dict = {}
    for luse in luses:
        
        year_dict = {}
        for chirts_year in range(2011, 2017):
            
            ds_tmax = xr.open_dataset(fr'data/common/chirts/chirts_{chirts_year}.nc').tmax
            ds_tmax['T'] = pd.date_range(datetime(chirts_year, 1, 1),
                                          datetime(chirts_year, 12, 31),
                                          freq='1d'
                                        )
            ds_tmax = ds_tmax.rio.write_crs(4326)
            ds_lu = retrieve_ghs(province=province).sel(time=lu_year)
            ds_tmax_clipped = clip_subroutine(ds_tmax,
                                              province,
                                              'X', 'Y')
            
            ds_threshold_days = xr.where(ds_tmax_clipped>=temp_threshold,
                                         1,
                                         0).sum(dim='T')
            ds_threshold_days, ds_lu_repr = regrid_match(ds_threshold_days,
                                                         ds_lu,
                                                         ds_threshold_days.rio.crs,
                                                         ds_lu.rio.crs,
                                                         'X', 'Y',
                                                         'x', 'y'
                                                        )
            ds_lu_repr = xr.where(ds_lu_repr<0,
                                  np.nan,
                                  ds_lu_repr
                                 )
            ds_lu_repr_class = classify_urban_rural(ds_lu_repr,
                                                    urban_tiles,
                                                    rural_tiles
                                                   ) 
            # interpolate
            Y_dir, X_dir = ds_lu_repr_class.indexes.values()
            ds_threshold = ds_threshold_days.interp(X=X_dir, 
                                                    Y=Y_dir, 
                                                    method="nearest"
                                                   ) # interp to model grid
            
            if luse == 'urban':
                class_ = 1
                anti = 0
            else:
                class_ = 0
                anti = 1
                
            threshold_days = xr.where(ds_lu_repr_class==class_,
                                            ds_threshold, np.nan
                                           )
            
            year_dict[chirts_year] = np.round(threshold_days.mean().values, 1)
            
        luse_dict[luse] = year_dict
    
    data_days_dict[province] = luse_dict

In [45]:
data_days_dict

{'istanbul': {'urban': {2011: 15.0,
   2012: 41.3,
   2013: 17.3,
   2014: 16.6,
   2015: 16.9,
   2016: 32.1},
  'nourban': {2011: 10.1,
   2012: 31.8,
   2013: 8.2,
   2014: 8.9,
   2015: 15.0,
   2016: 23.0}},
 'ankara': {'urban': {2011: 26.3,
   2012: 56.9,
   2013: 34.4,
   2014: 53.8,
   2015: 49.5,
   2016: 49.2},
  'nourban': {2011: 26.0,
   2012: 52.5,
   2013: 33.4,
   2014: 51.7,
   2015: 46.2,
   2016: 49.4}}}